In [24]:
import pandas as pd

df = pd.read_pickle("data_clean.pkl")
dict = {'Real': 0, 'Fake': 1}
df["Type"] = df["Type"].map(dict)
df

,Month,Text,Type
0,"Feb,20",kuala lumpur tourism art culture ministry focu...,0
1,"Feb,20",kuching sarawak record four new patient invest...,0
2,"Feb,20",johor baru police open investigation paper spr...,0
3,"Feb,20",johor baru malaysian love like red red rise bl...,0
4,"Feb,20",petaling jaya one day least love trump even an...,0
...,...,...,...
885,"Nov,20",najib loot rm billion bond epf pas rm billion ...,1
886,"Nov,20",regret inform kelantan state ministry health t...,1
887,"Nov,20",due influence majority malaysian face salary p...,1
888,"Nov,20",oldtown curry noodle restaurant rumor contain ...,1


In [25]:
X = df["Text"].values
y = df["Type"].values

In [26]:
#from sklearn.model_selection import train_test_split
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42, stratify=y)

In [27]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(min_df=0.01)

In [37]:
import pickle

pickle.dump(tfidf_vectorizer, open('tfidf_vectorizer.pkl', 'wb'))

In [28]:
#X_train_tf = tfidf_vectorizer.fit_transform(X_train)
X = tfidf_vectorizer.fit_transform(X)

In [29]:
#X_train_tf.shape
X.shape

(890, 1461)

In [30]:
#X_test_tf = tfidf_vectorizer.transform(X_test)

In [31]:
#X_train_tf

In [32]:
from sklearn.ensemble import RandomForestClassifier
rf_clf = RandomForestClassifier(n_estimators=1000, random_state=42)

In [33]:
rf_clf.fit(X, y)

RandomForestClassifier(n_estimators=1000, random_state=42)

In [44]:
rf_clf.predict_proba(X[0])

array([[0.969, 0.031]])

In [38]:
pickle.dump(rf_clf, open('random_forest.pkl', 'wb'))

In [ ]:
tfidf_pipeline.fit_transform(X)

In [8]:
from sklearn.naive_bayes import MultinomialNB
mnb_clf = MultinomialNB(alpha=0.84)

from sklearn.svm import SVC
svm_clf = SVC(kernel="rbf", C=9.0, gamma=0.00126)

from sklearn.tree import DecisionTreeClassifier
dt_clf = DecisionTreeClassifier(max_depth=5)

from sklearn.ensemble import RandomForestClassifier
rf_clf = RandomForestClassifier(n_estimators=1000, random_state=42)

from sklearn.linear_model import LogisticRegression
log_reg_clf = LogisticRegression(solver="lbfgs", random_state=42, C=96)

from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

ada_clf = AdaBoostClassifier(
    DecisionTreeClassifier(max_depth=1), n_estimators=200,
    algorithm="SAMME.R", learning_rate=0.63, random_state=42
)

In [9]:
from sklearn.ensemble import VotingClassifier
hard_voting_clf = VotingClassifier(
    estimators=[
        ('mnb', mnb_clf),
        ('svm', svm_clf),
        ('dt', dt_clf),
        ('rf', rf_clf),
        ('log_reg', log_reg_clf),
        ('adaboost', ada_clf)
    ],
    voting='hard'
)

In [10]:
from sklearn.model_selection import StratifiedKFold

kf = StratifiedKFold(n_splits=10)

In [11]:
%%time
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, roc_auc_score
import numpy as np
from scipy.stats import reciprocal
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import reciprocal, uniform

scores_mnb = {'train_acc':[], 'test_acc':[], 'train_f1':[], 'test_f1':[], 'train_precision':[], 'test_precision':[], 'train_recall':[], 'test_recall':[], 'train_auc':[], 'test_auc':[]}
scores_svm = {'train_acc':[], 'test_acc':[], 'train_f1':[], 'test_f1':[], 'train_precision':[], 'test_precision':[], 'train_recall':[], 'test_recall':[], 'train_auc':[], 'test_auc':[]}
scores_dt = {'train_acc':[], 'test_acc':[], 'train_f1':[], 'test_f1':[], 'train_precision':[], 'test_precision':[], 'train_recall':[], 'test_recall':[], 'train_auc':[], 'test_auc':[]}
scores_rf = {'train_acc':[], 'test_acc':[], 'train_f1':[], 'test_f1':[], 'train_precision':[], 'test_precision':[], 'train_recall':[], 'test_recall':[], 'train_auc':[], 'test_auc':[]}
scores_log_reg = {'train_acc':[], 'test_acc':[], 'train_f1':[], 'test_f1':[], 'train_precision':[], 'test_precision':[], 'train_recall':[], 'test_recall':[], 'train_auc':[], 'test_auc':[]}
scores_ada = {'train_acc':[], 'test_acc':[], 'train_f1':[], 'test_f1':[], 'train_precision':[], 'test_precision':[], 'train_recall':[], 'test_recall':[], 'train_auc':[], 'test_auc':[]}
scores_hard_voting = {'train_acc':[], 'test_acc':[], 'train_f1':[], 'test_f1':[], 'train_precision':[], 'test_precision':[], 'train_recall':[], 'test_recall':[], 'train_auc':[], 'test_auc':[]}

lookup_clf = {0:scores_mnb, 1:scores_svm, 2:scores_dt, 3:scores_rf, 4:scores_log_reg, 5:scores_ada}

param_distributions_mnb = {'alpha': np.linspace(0, 2, 20), 'fit_prior': [True, False]}
param_distributions_svm = {"gamma": reciprocal(0.001, 0.1), "C": uniform(1, 10)}
param_distributions_dt = {"max_depth": uniform(5, 100)}
param_distributions_rf = {"n_estimators": np.arange(5, 100)}
param_distributions_log = {"C": uniform(0.01, 100)}
param_distributions_ada = {"learning_rate": uniform(0, 2)}

param_distributions = [param_distributions_mnb, param_distributions_svm, param_distributions_dt, param_distributions_rf, param_distributions_log, param_distributions_ada]


for train_index, test_index in kf.split(X, y):
    X_train, X_test, y_train, y_test = X[train_index], X[test_index], y[train_index], y[test_index]
    clf_list = [mnb_clf, svm_clf, dt_clf, rf_clf, log_reg_clf, ada_clf]
    for index, clf in enumerate(clf_list):
        print(f"Training {clf.__class__.__name__}")
        clf.fit(X_train, y_train)
        rnd_search_cv = RandomizedSearchCV(clf, param_distributions[index], n_iter=20, verbose=0, cv=5)
        rnd_search_cv.fit(X_train, y_train)
        y_train_pred = rnd_search_cv.predict(X_train)
        #print("Training Accuracy :", clf.__class__.__name__, accuracy_score(y_train, y_train_pred))
        y_pred = rnd_search_cv.predict(X_test)
        #print("Validation Accuracy :", clf.__class__.__name__, accuracy_score(y_val, y_pred))
        #print("Validation F1-score :", clf.__class__.__name__, f1_score(y_val, y_pred))
        #print()
        train_accuracy = accuracy_score(y_train, y_train_pred)
        test_accuracy = accuracy_score(y_test, y_pred)
        
        train_f1 = f1_score(y_train, y_train_pred)
        test_f1 = f1_score(y_test, y_pred)
        
        train_precision = precision_score(y_train, y_train_pred)
        test_precision = precision_score(y_test, y_pred)
        
        train_recall = recall_score(y_train, y_train_pred)
        test_recall = recall_score(y_test, y_pred)
        
        train_roc_auc_score = roc_auc_score(y_train, y_train_pred)
        test_roc_auc_score = roc_auc_score(y_test, y_pred)
        
        scores_clf = lookup_clf[index]
        
        scores_clf["train_acc"].append(train_accuracy)
        scores_clf["test_acc"].append(test_accuracy)
        
        scores_clf["train_f1"].append(train_f1)
        scores_clf["test_f1"].append(test_f1)
        
        scores_clf["train_precision"].append(train_precision)
        scores_clf["test_precision"].append(test_precision)
        
        scores_clf["train_recall"].append(train_recall)
        scores_clf["test_recall"].append(test_recall)
        
        scores_clf["train_auc"].append(train_roc_auc_score)
        scores_clf["test_auc"].append(test_roc_auc_score)

Training MultinomialNB
Training SVC
Training DecisionTreeClassifier
Training RandomForestClassifier
Training LogisticRegression
Training AdaBoostClassifier
Training MultinomialNB


C:\Users\munch\anaconda3\envs\nlu\lib\site-packages\sklearn\naive_bayes.py:555: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  warnings.warn(
C:\Users\munch\anaconda3\envs\nlu\lib\site-packages\sklearn\naive_bayes.py:555: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  warnings.warn(
C:\Users\munch\anaconda3\envs\nlu\lib\site-packages\sklearn\naive_bayes.py:555: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  warnings.warn(
C:\Users\munch\anaconda3\envs\nlu\lib\site-packages\sklearn\naive_bayes.py:555: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  warnings.warn(
C:\Users\munch\anaconda3\envs\nlu\lib\site-packages\sklearn\naive_bayes.py:555: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  warnings.warn(


Training SVC
Training DecisionTreeClassifier
Training RandomForestClassifier
Training LogisticRegression
Training AdaBoostClassifier
Training MultinomialNB
Training SVC


C:\Users\munch\anaconda3\envs\nlu\lib\site-packages\sklearn\naive_bayes.py:555: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  warnings.warn(
C:\Users\munch\anaconda3\envs\nlu\lib\site-packages\sklearn\naive_bayes.py:555: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  warnings.warn(
C:\Users\munch\anaconda3\envs\nlu\lib\site-packages\sklearn\naive_bayes.py:555: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  warnings.warn(
C:\Users\munch\anaconda3\envs\nlu\lib\site-packages\sklearn\naive_bayes.py:555: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  warnings.warn(
C:\Users\munch\anaconda3\envs\nlu\lib\site-packages\sklearn\naive_bayes.py:555: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  warnings.warn(


Training DecisionTreeClassifier
Training RandomForestClassifier
Training LogisticRegression
Training AdaBoostClassifier
Training MultinomialNB
Training SVC


...

Training SVC
Training DecisionTreeClassifier
Training RandomForestClassifier
Training LogisticRegression
Training AdaBoostClassifier
Training MultinomialNB
Training SVC
Training DecisionTreeClassifier
Training RandomForestClassifier
Training LogisticRegression
Training AdaBoostClassifier
Training MultinomialNB


C:\Users\munch\anaconda3\envs\nlu\lib\site-packages\sklearn\naive_bayes.py:555: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  warnings.warn(
C:\Users\munch\anaconda3\envs\nlu\lib\site-packages\sklearn\naive_bayes.py:555: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  warnings.warn(
C:\Users\munch\anaconda3\envs\nlu\lib\site-packages\sklearn\naive_bayes.py:555: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  warnings.warn(
C:\Users\munch\anaconda3\envs\nlu\lib\site-packages\sklearn\naive_bayes.py:555: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  warnings.warn(
C:\Users\munch\anaconda3\envs\nlu\lib\site-packages\sklearn\naive_bayes.py:555: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  warnings.warn(


Training SVC
Training DecisionTreeClassifier
Training RandomForestClassifier
Training LogisticRegression
Training AdaBoostClassifier
Training MultinomialNB
Training SVC


C:\Users\munch\anaconda3\envs\nlu\lib\site-packages\sklearn\naive_bayes.py:555: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  warnings.warn(
C:\Users\munch\anaconda3\envs\nlu\lib\site-packages\sklearn\naive_bayes.py:555: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  warnings.warn(
C:\Users\munch\anaconda3\envs\nlu\lib\site-packages\sklearn\naive_bayes.py:555: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  warnings.warn(
C:\Users\munch\anaconda3\envs\nlu\lib\site-packages\sklearn\naive_bayes.py:555: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  warnings.warn(
C:\Users\munch\anaconda3\envs\nlu\lib\site-packages\sklearn\naive_bayes.py:555: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  warnings.warn(


Training DecisionTreeClassifier
Training RandomForestClassifier
Training LogisticRegression
Training AdaBoostClassifier
Wall time: 13min 42s


In [13]:
import statistics
clf_list = [mnb_clf, svm_clf, dt_clf, rf_clf, log_reg_clf, ada_clf, hard_voting_clf]
ls = ["train_acc", "test_acc"]
for index, scores_clf in lookup_clf.items():
    clf_class = clf_list[index].__class__.__name__
    print(f"{clf_class}")
    for key, item in scores_clf.items():
        mean = statistics.mean(item)
        print(f"{key}: {mean}")
    print()

MultinomialNB
train_acc: 0.9153558052434457
test_acc: 0.8876404494382022
train_f1: 0.9086408064577394
test_f1: 0.8744776447798082
train_precision: 0.9740150542671823
test_precision: 0.9623678446246512
train_recall: 0.8515151515151516
test_recall: 0.8045454545454546
train_auc: 0.9146464646464647
test_auc: 0.8867171717171717

SVC
train_acc: 0.9803995006242197
test_acc: 0.9730337078651685
train_f1: 0.9800506957511985
test_f1: 0.9725292237833162
train_precision: 0.9864597245733538
test_precision: 0.975517039486264
train_recall: 0.9737373737373737
test_recall: 0.9704545454545455
train_auc: 0.9803254769921437
test_auc: 0.9730050505050505

DecisionTreeClassifier
train_acc: 0.9903870162297128
test_acc: 0.9393258426966292
train_f1: 0.9901321332927338
test_f1: 0.9376197190052635
train_precision: 0.9984455194509165
test_precision: 0.9502703556890713
train_recall: 0.9820707070707071
test_recall: 0.9272727272727272
train_auc: 0.9902946127946128
test_auc: 0.9391919191919191

RandomForestClassifier
t

In [43]:
from sklearn.naive_bayes import MultinomialNB
mnb_clf = MultinomialNB(alpha=0.84)

mnb_clf.fit(X_train_tf, y_train)

MultinomialNB(alpha=0.84)

In [44]:
y_pred = mnb_clf.predict(X_test_tf)

In [45]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.82      0.99      0.90       135
           1       0.98      0.78      0.87       132

    accuracy                           0.88       267
   macro avg       0.90      0.88      0.88       267
weighted avg       0.90      0.88      0.88       267



In [27]:
df_idf = pd.DataFrame(tfidf_vectorizer.idf_, index=tfidf_vectorizer.get_feature_names(),columns=["idf_weights"])

C:\Users\munch\anaconda3\envs\nlu\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [20]:
df_idf

,idf_weights
abdul,3.909790
abdullah,3.341806
accord,3.939643
act,3.747271
action,3.410799
...,...
worker,3.504325
would,2.248765
yaakob,3.798564
year,2.903551
